In [1]:
#Libraries used in the functions below
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import math

from jupyter_dash import JupyterDash
import os
try:
    os.environ.pop('http_proxy')
    os.environ.pop('https_proxy')
except KeyError:
    pass

# Import dash
import dash
from dash import Dash, html, dcc, Input, Output, dash_table, State

# Import plotly
import plotly.graph_objs as go
import plotly.express as px
import seaborn as sns
pd.options.mode.chained_assignment = None 

In [2]:
#Datasets turned into Pandas DataFrames
dfC = pd.read_csv("dataset/courses.csv")
dfSA=pd.read_csv("dataset/studentAssessment.csv")
dfSI=pd.read_csv("dataset/studentInfo.csv")
dfSR=pd.read_csv("dataset/studentRegistration.csv")
dfVLE=pd.read_csv("dataset/vle.csv")
dfSVLE=pd.read_csv("dataset/studentVle.csv")
dfA=pd.read_csv("dataset/assessments.csv")
dfP=pd.read_csv("dataset/generalized_predicts.csv")

In [3]:
dfC['Course_Section'] = dfC[['code_module', 'code_presentation']].apply(lambda x: '-'.join(x), axis=1)
course_section=list(dfC['Course_Section'] )
dfP['Course_Section'] = dfP[['code_module', 'code_presentation']].apply(lambda x: '-'.join(x), axis=1)
course_section=course_section[:-3]
course_section

['AAA-2013J',
 'AAA-2014J',
 'BBB-2013J',
 'BBB-2014J',
 'BBB-2013B',
 'BBB-2014B',
 'CCC-2014J',
 'CCC-2014B',
 'DDD-2013J',
 'DDD-2014J',
 'DDD-2013B',
 'DDD-2014B',
 'EEE-2013J',
 'EEE-2014J',
 'EEE-2014B',
 'FFF-2013J',
 'FFF-2014J',
 'FFF-2013B',
 'FFF-2014B']

In [4]:
dfSI['Course_Section'] = dfSI[['code_module', 'code_presentation']].apply(lambda x: '-'.join(x), axis=1)
def roster(course_section,sort_type):
    dfRoster=pd.merge(dfSI,dfP, on=['id_student','Course_Section'])
    dfClassRoster=dfRoster[dfRoster['Course_Section']==course_section]
    if sort_type=='Numerical':
        dfClassRoster=dfClassRoster.sort_values('id_student')
    else:
        dfClassRoster=dfClassRoster.sort_values('probability',ascending=False)
    return list(dfClassRoster['id_student'])

In [5]:
df_svle_with_type = dfSVLE.merge(dfVLE[['id_site','activity_type']], left_on='id_site', right_on='id_site')
df_svle_with_type.head()

,code_module,code_presentation,id_student,id_site,date,sum_click,activity_type
0,AAA,2013J,28400,546652,-10,4,forumng
1,AAA,2013J,28400,546652,-10,1,forumng
2,AAA,2013J,28400,546652,-10,1,forumng
3,AAA,2013J,28400,546652,-10,8,forumng
4,AAA,2013J,30268,546652,-10,3,forumng


In [6]:
# create svle_with_type_dict dictionary by class
# get all combinations of code_module and code_presentation
df_svle_with_type['Course_Section'] = df_svle_with_type['code_module']+'-'+df_svle_with_type['code_presentation']
combos = (df_svle_with_type['Course_Section']).unique()
svle_with_type_dict = {}
for combo in combos:
    svle_with_type_dict[combo] = df_svle_with_type[df_svle_with_type['Course_Section']==combo]

In [7]:
dfA['Course_Section'] = dfA[['code_module', 'code_presentation']].apply(lambda x: '-'.join(x), axis=1)
resource_dict = dict(zip(dfVLE['id_site'], dfVLE['activity_type']))
def classCode(Module,Presentation):
    dfCode=dfSVLE[dfSVLE['code_module']==Module]
    dfCode=dfCode[dfCode['code_presentation']==Presentation]
    return dfCode
dfSVLE["id_site"]=dfSVLE['id_site'].map(resource_dict)

In [8]:
df_sa_with_due_dates = dfSA.merge(dfA[['id_assessment','date','code_module','code_presentation','weight']], left_on='id_assessment', right_on='id_assessment')
#dfSI[['code_module', 'code_presentation']].apply(lambda x: '-'.join(x), axis=1)
df_sa_with_due_dates['Course_Section'] = df_sa_with_due_dates[['code_module','code_presentation']].apply(lambda x: '-'.join(x), axis=1)
df_si_with_preds =pd.merge(dfSI,dfP, on=['id_student','Course_Section'])
student_info_table_cols = ['id_student','Predicted Fail/Withdraw %', '# of Assignments Completed','Current Grade (weighted)', "Avg of this student's previous scores",'Most recent score',"Most recent score - this student's previous avg"]

In [33]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

# Create a unique list of code_module
available_indicators1 = course_section

# Step 1

app.layout = html.Div([
                        html.H1("Instructor Interface/Dashboard",style={'textAlign':'center', 'paddingTop':'1%','marginBottom':'-16px'}),
                        html.Div([
                                html.Div(["Module-presentation:"], style={'float':'left','margin':'0px','display':'inline-block','fontSize':'20px','fontWeight':'light'}),
                                html.Div(
                                    [dcc.Dropdown(
                                        id='CourseSection',
                                        options=[{'label': i, 'value': i} for i in available_indicators1],
                                        value=available_indicators1[0],
                                        #style={'display':'inline-block','float':'left'}
                                    )],style={'minWidth':'150px','display':'inline-block'}
                                ),
                                
                            ],style={'display':'inline-block','marginTop':'-40px'}),
                        html.Hr(style={'marginTop':'0px'}),
                        html.Div([
                            html.H5(id='all_student_table_title'),
                            dash_table.DataTable(
                                id="student_info_table",
                                columns=[{"name": i, "id": i} for i in student_info_table_cols],
                                style_cell={
                                        'minWidth': '90px', 
                                        'width': '90px',
                                        'maxWidth': '90px',
                                        'overflow': 'hidden',
                                        'textOverflow': 'ellipsis'},
                                fixed_rows={'headers': True},
                                style_table={'height':'300px'},
                                style_data_conditional=[{
                                    'if':{'row_index':'odd'},
                                    'backgroundColor':'rgb(235,235,235)',
                                },
                                {
                                    'if':{
                                        'filter_query':'{Predicted Fail/Withdraw %} > 0.8'
                                    },
                                    'backgroundColor':'rgb(255,204,203)',
                                    'fontWeight':'bold'
                                },
                                {
                                    'if':{
                                        'filter_query':"{Most recent score - this student's previous avg} > 10",
                                        'column_id':"Most recent score - this student's previous avg"
                                    },
                                    'backgroundColor':'rgb(187,255,153)'
                                },
                                {
                                    'if':{
                                        'filter_query':"{Most recent score - this student's previous avg} < -10",
                                        'column_id':"Most recent score - this student's previous avg"
                                    },
                                    'backgroundColor':'rgb(255,128,128)'
                                },
                                {
                                    'if':{
                                        'filter_query':"{id_student} = No"
                                    },
                                    'backgroundColor':'rgb(255,204,0)',
                                    'fontWeight':'bold'
                                }
                                ],
                                style_header={'whiteSpace':'normal'},
                                filter_action="native",
                                sort_action="native",
                                sort_mode="multi",
                                data=[]
                                ),
                            dcc.RadioItems(
                                ['Assessment', 'Resource Usage (Total)','Resource Usage over Time'],
                                'Assessment',
                                id='Graph-Type',
                                inline=True),
                            dcc.Graph(id='GraphMean'),
                            
                            #dcc.Graph(id='GraphMedian'),
                            html.H6("Days Elapsed:"),
                            dcc.Slider(
                                    0,
                                    200,
                                    #step=None,
                                    tooltip={'always_visible':True},
                                    id='Day',
                                    value=60),
                        html.Div(id="final_table")],style={'width': '48%','display': 'inline-block'}),
                        html.Div([
                            html.Div(id='student_id_msg'),
                            html.H6(id='single_student_table_title'),
                            html.Div(id="Student_table"),
                            html.Div(id="Student_Comments"),

                        ], style={'width': '48%','display': 'inline-block','float': 'right'})
                ], style={'width':'95%','margin':'auto'}
            )
@app.callback(
    Output('student_info_table','active_cell'),
    Output('student_info_table','selected_cells'),
    Input('CourseSection','value')
)
def reset_selection(course_section):
    return None, []

@app.callback(
    Output('all_student_table_title','children'),
    #Input('CourseSection','value'),
    Input('Day','value')
)
def set_all_student_table_title(day):
    return "Early Warning Table as of day {0}".format(day)


@app.callback(
    Output('Day','max'),
    Output('Day','marks'),
    Input('CourseSection','value')
)
def set_slider_max(course_section):
    if not course_section:
        return None, None
    course_assignments = dfA[dfA['Course_Section']==course_section].dropna()
    assignments = pd.DataFrame(course_assignments[['id_assessment','date']].sort_values(by='date'))
    assignments['number'] = range(1,len(assignments)+1)
    marks = {int(x['date']):{'label':'Assignment\u00A0{0}'.format(x['number']), 'style':{'color':'black'}} for x in assignments.to_dict('records')}
    marks[0] = {'label':'0'}
    max_date = np.max(course_assignments['date'])
    marks[int(max_date+14)] = {'label':'{0}'.format(int(max_date+14))}
    return [max_date+14,marks]


@app.callback(
    Output('single_student_table_title','children'),
    Input('student_info_table','active_cell'),
    Input('student_info_table','derived_viewport_data')
)
def set_single_student_table_title(active_cell, data):
    id_student = get_id_student(active_cell, data)
    if id_student is not None:
        return "Student {0}'s Information".format(id_student)
    return None

def get_id_student(active_cell, data):
    if active_cell:
        return data[active_cell['row']]['id_student']
    elif data:
        return data[0]['id_student']
    else:    
        return None

@app.callback(
    Output('student_id_msg', 'children'),
    Input('student_info_table','active_cell'),
    Input('student_info_table','derived_viewport_data'),
    )
def set_id_value(active_cell, data):
    id_student = get_id_student(active_cell, data)
    if id_student is not None:
        return "Selected student: {0}. Select another student by clicking on the Early Warning Table.".format(id_student)
    return "No data availabe - remove filters from the Early Warning Table!"

@app.callback(
    Output('student_info_table', 'data'),
    Input('CourseSection', 'value'),Input('Day','value')
)
def update_student_info_table(course_section, day):
    if not course_section:
        to_ret = pd.DataFrame([["No","Course","Selected","-----","Select","Course","Above"]], columns=student_info_table_cols)
        return to_ret.to_dict('records')

    df_roster_this_course = pd.DataFrame(df_si_with_preds[df_si_with_preds['Course_Section']==course_section])
    
    # select only student assessments from this course, join with roster to guarantee students who don't have 
    # any assignments turned in still show up.
    df_sa_this_course = df_sa_with_due_dates[df_sa_with_due_dates['Course_Section']==course_section]
    df_student_assessments = df_roster_this_course.merge(df_sa_this_course, left_on='id_student',right_on='id_student')

    # filter out assignments due after day
    df_student_assessments = df_student_assessments[df_student_assessments.date <= day]
    # filter out assignments submitted after day
    df_student_assessments = df_student_assessments[df_student_assessments.date_submitted <= day]

    # calculate total possible scores:
    total_possible_score = dfA[dfA.date <= day]
    total_possible_score = total_possible_score[total_possible_score['Course_Section'] == course_section]
    # only count TMAs
    total_possible_score = np.sum(total_possible_score[total_possible_score['assessment_type']=='TMA']['weight'])

    # calculate each student's scores
    df_student_assessments['weighted_score'] = df_student_assessments['score'] * df_student_assessments['weight']
    df_student_scores = df_student_assessments.groupby(['id_student']).sum().reset_index()[['id_student','weighted_score']]
    df_student_scores['pct_total'] = np.round(df_student_scores['weighted_score']/total_possible_score,1)

    # get stats about all students most recent scores
    most_rec_scores = get_most_recent_assignment_scores(dfSA, dfA, dfSI, course_section, day)

    no_assignments_submitted_msg = "No prior assignments submitted"
    
    most_rec_scores.loc[most_rec_scores['num_assignments_submitted']>1, 'avg_before'] = np.round(
        (most_rec_scores.loc[most_rec_scores['num_assignments_submitted']>1, 'stu_avg_score']*most_rec_scores.loc[most_rec_scores['num_assignments_submitted']>1, 'num_assignments_submitted'] -
        most_rec_scores.loc[most_rec_scores['num_assignments_submitted']>1, 'most_rec_score'])/(most_rec_scores.loc[most_rec_scores['num_assignments_submitted']>1, 'num_assignments_submitted'] -1)
        ,1)
    most_rec_scores.loc[most_rec_scores['num_assignments_submitted']>1, 'recent_minus_avg'] = np.round(
        most_rec_scores.loc[most_rec_scores['num_assignments_submitted']>1, 'most_rec_score'] - most_rec_scores.loc[most_rec_scores['num_assignments_submitted']>1, 'avg_before'],1)
    
    most_rec_scores.loc[most_rec_scores['num_assignments_submitted']<=1,'avg_before'] = no_assignments_submitted_msg
    no_prev_avg_msg = 'No prev. avg'
    most_rec_scores.loc[most_rec_scores['num_assignments_submitted']<=1, 'recent_minus_avg'] = no_prev_avg_msg
    # add column indicating how many asisgnments turned in vs total
    most_rec_scores['assignments_completed_frac'] = most_rec_scores['num_assignments_submitted'].astype(str) + '/' + most_rec_scores['num_assignments_due'].astype(str)

    num_assignments_due = np.max(most_rec_scores['num_assignments_due'])
    if pd.isna(num_assignments_due):
        num_assignments_due = 0

    # prepare final df for output
    df_final = df_roster_this_course.merge(df_student_scores, how='left', left_on='id_student',right_on='id_student')[['id_student','probability','pct_total']]
    df_final = df_final.merge(most_rec_scores, how='left',left_on='id_student',right_on='id_student')[list(df_final.columns) + ['avg_before','most_rec_score', 'assignments_completed_frac', 'num_assignments_submitted','recent_minus_avg']]

    # select final columns desired
    df_final = df_final[['id_student','probability','assignments_completed_frac','pct_total','avg_before','most_rec_score','recent_minus_avg']]
    #df_final['recent_minus_avg'] = df_final['most_rec_score'] - df_final['avg_before']
    
    # fill-in blanks
    df_final.loc[df_final['pct_total'].isna(),['assignments_completed_frac','pct_total','avg_before','most_rec_score','recent_minus_avg']] = [
        '0/{0}'.format(num_assignments_due),0,no_assignments_submitted_msg,"Nothing submitted",no_prev_avg_msg
    ]


    # if less than 2 assignments turned in - don't show predictions
    if num_assignments_due < 2:
        df_final['probability'] = 'Too early to predict'
    
    df_final.columns = student_info_table_cols
    return df_final.to_dict("records")

    
def get_most_recent_assignment_scores(df_sa, df_a, df_si, course_section, day):
    
    # filter out student assessments submitted on day or after
    df_sa_filtered = df_sa[df_sa['date_submitted'] <= day]

    # change any NaNs in student assessment table to 0
    df_sa_filtered.loc[df_sa_filtered['score'].isna(),['score']] = 0
    
    # join with student info - in case student has not submitted any assignments yet...
    student_info = df_si[df_si['Course_Section']==course_section]
    df_sa_filtered = student_info[['id_student']].merge(df_sa_filtered, how='left',left_on='id_student',right_on='id_student')
    
    # filter out assignments with due date after day
    df_a_filtered = df_a[(df_a['date']<=day) & (df_a['Course_Section']==course_section)]
    
    # calc total # of assignments due
    num_assignments_submittable = df_a_filtered['id_assessment'].count()

    # join to remove any early-submitted assignments from being shown
    df_merged = df_sa_filtered.merge(df_a_filtered, left_on='id_assessment', right_on='id_assessment').sort_values(by=['date_submitted','date'],ascending=[False,False])
    
    # count the number of assignments turned in by student
    num_assignments = df_merged.groupby(['id_student'])['id_assessment'].count().reset_index()[['id_student','id_assessment']].rename(columns={'id_assessment':'num_assignments_submitted'})
    
    # return the average for all assignments turned in by student
    assignment_avg = df_merged.groupby(['id_student'])['score'].mean().reset_index()[['id_student','score']].rename(columns={'score':'stu_avg_score'})

    # get the score of last assignment turned in by the student (for comparison against their previous average)
    to_ret = df_merged.loc[df_merged.groupby(['id_student'])['date_submitted'].idxmax(),['id_student','id_assessment','score']].rename(columns={'score':'most_rec_score'})

    # combine to single df
    to_ret = to_ret.merge(num_assignments, left_on='id_student', right_on='id_student')[['id_student','id_assessment','most_rec_score','num_assignments_submitted']]
    to_ret = to_ret.merge(assignment_avg, left_on='id_student', right_on='id_student')[list(to_ret.columns) + ['stu_avg_score']]
    to_ret['num_assignments_due'] = num_assignments_submittable
    return to_ret



@app.callback(
    Output('GraphMean', 'figure'),
    Input('CourseSection', 'value'),
    Input('student_info_table','active_cell'),
    Input('student_info_table','derived_viewport_data'),
    Input('Graph-Type','value'),
    Input('Day','value'))
def update_figure(CourseSection1, active_cell, data,type1,day1): 
    if not CourseSection1:
        return px.bar() 
    StudentID1 = get_id_student(active_cell, data)
    if type1=='Assessment':       
        assessments = df_sa_with_due_dates.loc[(df_sa_with_due_dates['date']<=day1) & (df_sa_with_due_dates['Course_Section']==CourseSection1),['id_assessment','id_student','score','date_submitted']]
        assessment_avg = assessments[['id_assessment','score']].groupby('id_assessment').mean().reset_index()
        assessment_avg['type'] = 'Mean'

        dfStudentScore = assessments.loc[(assessments['id_student']==StudentID1) & (assessments['date_submitted']<=day1),['id_assessment','score']]
        dfStudentScore['type'] = 'Student'

        dfTogether=pd.concat([dfStudentScore, assessment_avg], ignore_index=True,axis=0)
        dfTogether['id_assessment'] = dfTogether.id_assessment.astype(str)
        dfTogether=dfTogether.dropna()
        figure = px.bar(dfTogether, x="id_assessment", y="score", color="type", barmode="group",
                title="Comparison of Students Scores and Class Averages")
        figure.update_layout(legend_title="")
        return figure
    elif type1=='Resource Usage (Total)':
        dfSVLE1 = svle_with_type_dict[CourseSection1]
        dfCode = dfSVLE1[dfSVLE1['date']<=day1]
        student_count=len(dfCode['id_student'].unique())

        dfResource=dfCode.groupby(['activity_type']).sum()
        dfResource['sum_click']=dfResource['sum_click']/student_count
        dfResource=dfResource.reset_index()
        dfStudent=dfCode[dfCode['id_student']==StudentID1]
        
        dfStudent=dfStudent.groupby(['activity_type']).sum()
        dfStudent=dfStudent.reset_index()
        dfStudent['type']='Student'
        dfResource['type']='Mean'
        dfTogether1=pd.concat([dfStudent, dfResource], ignore_index=True,axis=0)
        figure2 = px.bar(dfTogether1, x="activity_type", y="sum_click", color="type", barmode="group",
            title="Comparison of Students Resource Usage and Class Averages")
        figure2.update_layout(legend_title="")
        return figure2
    else:
        return update_figure_median(active_cell, data, CourseSection1, day1)

# @app.callback(
#     Output('GraphMedian', 'figure'),
#     Input('student_info_table','active_cell'),
#     Input('student_info_table','derived_viewport_data'),
#     Input('CourseSection', 'value'),
#     Input('Day','value'))
def update_figure_median(active_cell, data,course_section,day1): 
    if not course_section:
        return px.line()
    StudentID1 = get_id_student(active_cell, data)
    current=14
    dfSVLE_this_course = svle_with_type_dict[course_section]
    dfSVLE_this_course = dfSVLE_this_course[dfSVLE_this_course['id_student']==StudentID1]
    dfSVLE1=dfSVLE_this_course[(dfSVLE_this_course['date']>=0) & (dfSVLE_this_course['date']<=14)]
    dfClass=dfSVLE1.groupby(['activity_type','id_student']).sum()
    dfClass['date']=14
    dfStart=dfClass.reset_index()
    while current+14<day1:
        dfSVLE1=dfSVLE_this_course[(dfSVLE_this_course['date']>current) & (dfSVLE_this_course['date']<=current+14)]
        dfClass=dfSVLE1.groupby(['activity_type']).sum()
        dfClass['date']=current+14
        dfClass=dfClass.reset_index()
        dfStart=pd.concat([dfStart,dfClass], ignore_index=True,axis=0)
        current=current+14
    fig = px.line(dfStart, x="date", y="sum_click", color='activity_type', title='Resource Access')
    return fig


@app.callback(
    Output('final_table', 'children'),Input('CourseSection', 'value'),Input('Graph-Type','value'),Input('Day','value'))

def update_figure(CourseSection1, type1, day1):
    if type1=='Assessment':
        dfA1=dfA[dfA['date']<=day1]
        dfCourse_Assessments=dfA1[dfA1['Course_Section']==CourseSection1]
        assessment=list(dfCourse_Assessments['id_assessment'])
        trial1=[]
        for assess in assessment:
            dfSAScore=dfSA[dfSA['id_assessment']==assess]
            trial1.append([assess]+list(dfSAScore['score'].describe().round(1)))
        data1=pd.DataFrame(trial1,columns=['Assessment','Count','Mean','STD','Min','25%','50%','75%','Max'])
        return [dash_table.DataTable(
                id='Summary',
                columns=[{"name": i, "id": i} for i in data1.columns],
                data=data1.to_dict('records'),
            )]
    else:
        dfCode = svle_with_type_dict[CourseSection1]
        dfClass=dfCode.groupby(['activity_type','id_student']).sum()
        dfClass=dfClass.reset_index()
        resourceType=dfClass['activity_type'].unique()
        trial2=[]
        for idSite in resourceType:
            dfClick=dfClass[dfClass['activity_type']==idSite]
            trial2.append([idSite]+list(dfClick['sum_click'].describe().round(1)))
        data2=pd.DataFrame(trial2,columns=['Resource','Count','Mean','STD','Min','25%','50%','75%','Max'])
        return [dash_table.DataTable(
                id='Summary',
                columns=[{"name": i, "id": i} for i in data2.columns],
                data=data2.to_dict('records'),
            )]

@app.callback(
    [Output('Student_table', 'children'),
     Output('Student_Comments', 'children')],
    [Input('CourseSection', 'value'),
     Input('student_info_table','active_cell'),
     Input('student_info_table','derived_viewport_data'),
     Input('Day','value')
    ])
def update_figure(CourseSection1, active_cell, data, day1):
    if not CourseSection1:
        to_ret= pd.DataFrame([["Select","Course","From","Dropdown!"]], columns=['Assessment/Resource','Student Score/Clicks','Difference (Sum_Click - Median)','Difference (Sum_Click - Mean)'])
        return [dash_table.DataTable(columns=[{"name":i,"id":i} for i in to_ret.columns], data=to_ret.to_dict('records')),None]
    StudentID1 = get_id_student(active_cell, data)
        
    # student score df is: 
    # df of assessments with data about score, different from median, and difference from mean
    # get list of all assessments (whether or not this student has completed them)
    assessments = df_sa_with_due_dates[(df_sa_with_due_dates['date']<=day1) & (df_sa_with_due_dates['Course_Section']==CourseSection1)]
    assessment_ids = dfA.loc[(dfA['date']<= day1) & (dfA['Course_Section']==CourseSection1),['id_assessment']]
    
    students_assessments = assessments[(assessments['id_student']==StudentID1) &(assessments['date_submitted']<=day1)]
    dfStudentScore = assessment_ids.merge(students_assessments, how='left',left_on='id_assessment',right_on='id_assessment')
    dfStudentScore['score'].fillna(0)

    assessment_aggs = assessments.groupby('id_assessment').agg({'score':['mean','std','median']})
    assessment_aggs.columns = ['Mean','STDEV','Median']
    assessment_aggs = assessment_aggs.reset_index()

    dfStudentScore = dfStudentScore.merge(assessment_aggs, how='right',left_on='id_assessment',right_on='id_assessment')
    
    # fill in nan with zeros
    dfStudentScore=dfStudentScore.fillna(0)
    dfStudentScore['DifferenceMean']=(dfStudentScore['score']-dfStudentScore['Mean']).round(1)
    dfStudentScore['DifferenceMedian']=(dfStudentScore['score']-dfStudentScore['Median']).round(1)
    dfStudentScore=dfStudentScore[['id_assessment','score','DifferenceMedian','DifferenceMean','STDEV']]
    dfStudentScore.columns=['ID','Student','Difference (Sum_Click - Median)','Difference (Sum_Click - Mean)','STDEV']
    dfStudentScore=dfStudentScore.fillna(0)

    #SVLE section
    dfSVLE1 = svle_with_type_dict[CourseSection1]
    dfSVLE1 = dfSVLE1[dfSVLE1['date']<=day1]
    
    # use groupby and aggregate functions
    # first sum up by each student and resource type
    svle_student_sums = svle_with_type_dict[CourseSection1][['activity_type','id_student','sum_click']].groupby(['id_student','activity_type']).sum()
    svle_student_sums = svle_student_sums.reset_index()
    svle_aggs = svle_student_sums.groupby('activity_type').agg({'sum_click':['mean','std','median']})
    svle_aggs.columns = ['Mean','STDEV','Median']
    svle_aggs = svle_aggs.reset_index()
    
    dfStudent = svle_student_sums[svle_student_sums['id_student']==StudentID1]
    dfStudent = svle_aggs.merge(dfStudent, how='left',left_on='activity_type',right_on='activity_type')
    
    # select desired columns
    dfStudentResource=dfStudent.fillna(0)[['activity_type','sum_click','Mean','STDEV','Median']]
    
    dfStudentResource['Median']=(dfStudentResource['sum_click']-dfStudentResource['Median']).round(1)
    dfStudentResource['Mean']=(dfStudentResource['sum_click']-dfStudentResource['Mean']).round(1)
    dfStudentResource=dfStudentResource[['activity_type','sum_click','Median','Mean','STDEV']]
    dfStudentResource.columns=['ID','Student','Difference (Sum_Click - Median)','Difference (Sum_Click - Mean)','STDEV']

    # combine single student assessment and student vle data
    dfTogetherStudent=pd.concat([dfStudentScore, dfStudentResource], ignore_index=True,axis=0)
    count=0
    comment=[]
    text=""
    for thing in dfTogetherStudent['STDEV']:
        if dfTogetherStudent['Difference (Sum_Click - Mean)'][count]>0:
            if dfTogetherStudent['Difference (Sum_Click - Mean)'][count]<=0.5*thing:
                comment.append('Average')
            elif dfTogetherStudent['Difference (Sum_Click - Mean)'][count]>=2*thing:
                comment.append('Well Above Average')
            elif dfTogetherStudent['Difference (Sum_Click - Mean)'][count]>0.5*thing:
                comment.append('Above Average')
            count=count+1
        else:
            if -1*dfTogetherStudent['Difference (Sum_Click - Mean)'][count]<=0.5*thing:
                comment.append('Average')
            elif -1*dfTogetherStudent['Difference (Sum_Click - Mean)'][count]>=2*thing:
                comment.append('Well Below Average')
                if type(dfTogetherStudent['ID'][count])==type(14):
                    text=text+"This student performed Well Below Average on the "+str(dfTogetherStudent['ID'][count])+" assessment.\n"
                    text=text+"It is recommended that the student review the content related to the "+str(dfTogetherStudent['ID'][count])+" assessment, perform extra practice, and make appropriate corrections to their assessment.\n\n"
                else:
                    text=text+"This student's level of access to the "+str(dfTogetherStudent['ID'][count])+" resource was Well Below Average.\n"
                    text=text+"If the student is struggling with the course content, it is recommended that they spend more time accessing "+str(dfTogetherStudent['ID'][count])+".\n\n"

            elif -1*dfTogetherStudent['Difference (Sum_Click - Mean)'][count]>0.5*thing:
                comment.append('Below Average')
                if type(dfTogetherStudent['ID'][count])==type(14):
                    text=text+"This student performed Below Average on the "+str(dfTogetherStudent['ID'][count])+" assessment.\n"
                    text=text+"It is recommended that the student review the content related to the "+str(dfTogetherStudent['ID'][count])+" assessment, perform extra practice, and make appropriate corrections to their assessment.\n\n"
                else:
                    text=text+"This student's level of access to the "+str(dfTogetherStudent['ID'][count])+" resource was Below Average.\n"
                    text=text+"If the student is struggling with the course content, it is recommended that they spend more time accessing "+str(dfTogetherStudent['ID'][count])+".\n\n"            
            count=count+1
    dfTogetherStudent['STDEV']=comment
    dfTogetherStudent.columns=['Assessment/Resource','Student Score/Clicks','Difference (Sum_Click - Median)','Difference (Sum_Click - Mean)','Performance']
    dfTogetherStudent=dfTogetherStudent[['Assessment/Resource','Student Score/Clicks','Difference (Sum_Click - Median)','Difference (Sum_Click - Mean)']]

    return [dash_table.DataTable(
        id='Summary1',
        columns=[{"name": i, "id": i} for i in dfTogetherStudent.columns],
        data=dfTogetherStudent.to_dict('records'),
        )],[dcc.Textarea(
        id='textarea-example',
        value=text,
        style={'width': '100%','maxWidth':'100%', 'height': 300},
        )]


    
    
app.run_server(mode="external", port = 8401,debug=True)

Dash app running on http://127.0.0.1:8401/


In [20]:
set_slider_max('FFF-2013J')

[250.0,
 {19: {'label': 'Assignment\xa01', 'style': {'color': 'black'}},
  47: {'label': 'Assignment\xa02', 'style': {'color': 'black'}},
  96: {'label': 'Assignment\xa03', 'style': {'color': 'black'}},
  131: {'label': 'Assignment\xa04', 'style': {'color': 'black'}},
  173: {'label': 'Assignment\xa05', 'style': {'color': 'black'}},
  236: {'label': 'Assignment\xa013', 'style': {'color': 'black'}},
  0: {'label': '0'},
  250: {'label': '250'}}]

0       TMA
1       TMA
2       TMA
3       TMA
4       TMA
       ... 
201     CMA
202     TMA
203     TMA
204     TMA
205    Exam
Name: assessment_type, Length: 206, dtype: object